In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/EvaluationTSE


/content/drive/MyDrive/EvaluationTSE


In [ ]:
####!git clone https://github.com/tech-srl/code2vec #

In [3]:
cd code2vec

/content/drive/MyDrive/EvaluationTSE/code2vec


In [ ]:
# preprocessing

In [4]:
number_of_samples = 0

c = 0

with open('/content/drive/MyDrive/EvaluationTSE/code2vec/generated.java','r') as dataset_input: 
    with open('/content/drive/MyDrive/EvaluationTSE/code2vec/oracle.java','r') as dataset_output:
        with open('generated2.java','w') as input_file: 
            with open('oracle2.java','w') as output_file:
                temp_output = dataset_output.readlines()
                for i,line in enumerate(dataset_input):
                    number_of_samples += 1
                    if line.endswith('\t'):
                        c = c + 1
                        continue
                  
                    sum = line.count("public ") + line.count("private ") + line.count("protected ")
                    if sum < 2:
                        input_file.write(line)
                        output_file.write(temp_output[i])
                    else:
                        c = c + 1
                        #print(i)

remained_number_of_samples = number_of_samples - c

print('===================================')
print("Total Samples that removed = ",c)
print('===================================')
print("Total Samples that remain  = ", (remained_number_of_samples))
print('===================================')

Total Samples that removed =  1
Total Samples that remain  =  2


In [5]:
# Preprocessing on Methods in input(methods).java:

import subprocess


with open("generated2.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("generated2_TEMP.java", "w", encoding='utf-8') as f:
    for line in lines:
        f.write(line)


path = 'generated2_TEMP.java'
max_path_length = 8
max_path_width = 2
jar_path = 'JavaExtractor/JPredict/target/JavaExtractor-0.0.1-SNAPSHOT.jar'


error_indexes = []
flag_samples_remain = True

while remained_number_of_samples:
    command = ['java', '-cp', jar_path, 'JavaExtractor.App', '--max_path_length',
                    str(max_path_length), '--max_path_width', str(max_path_width), '--file', path, '--no_hash']

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    output = out.decode().splitlines()


    if len(output) == 0 and flag_samples_remain:
        #print(err)
        err = err.decode()
        num = ""
        read_int = err.find("at line ") + 8

        while err[read_int] != ",":
            num = num + err[read_int]
            read_int = read_int + 1

        num = int(num)

        if error_indexes:
            error_indexes.append(num + error_indexes[-1])
        else:
            error_indexes.append(num - 1)

        print("Error Index = " , error_indexes[-1])

        with open("generated2.java", "r", encoding='utf-8') as f:
            lines = f.readlines()

        with open("generated2_TEMP.java", "w", encoding='utf-8') as f:
            for i,line in enumerate(lines):
                if i > (error_indexes[-1]):
                    f.write(line)

        if error_indexes[-1] == (remained_number_of_samples - 1):
            flag_samples_remain = False  

    else:
        remained_number_of_samples = remained_number_of_samples - len(error_indexes)
        print('===================================')
        print("Total Samples that removed = ",len(error_indexes))
        print('===================================')
        print("Total Samples that remain = ",remained_number_of_samples)
        print('===================================')
        break

for i in range(len(error_indexes)):
   error_indexes[i] = error_indexes[i]


with open("generated2.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("generated2.java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line) 


with open("oracle2.java", "r", encoding='utf-8') as f:
    lines = f.readlines()

with open("oracle2.java", "w", encoding='utf-8') as f:
    for i,line in enumerate(lines):
        if i not in error_indexes:
            f.write(line)    

Error Index =  1
Total Samples that removed =  1
Total Samples that remain =  1


In [6]:
#with open('output(tests).java','r', encoding='utf-8') as firstfile, open('Input.java','w', encoding='utf-8') as secondfile:
with open('generated2.java','r', encoding='utf-8') as firstfile, open('Input.java','w', encoding='utf-8') as secondfile:
  for i,line in enumerate(firstfile):
    secondfile.write(line)

In [ ]:
#!wget https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz

In [ ]:
#!tar -xvzf java14m_model.tar.gz #

In [7]:
!rm  code_vector_results.txt

rm: cannot remove 'code_vector_results.txt': No such file or directory


In [ ]:
!python code2vec.py --load models/java14_model/saved_model_iter8.release --predict --export_code_vectors

2021-11-21 00:01:36.032686: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-21 00:01:36,034 INFO     
2021-11-21 00:01:36,034 INFO     
2021-11-21 00:01:36,034 INFO     ---------------------------------------------------------------------
2021-11-21 00:01:36,034 INFO     ---------------------------------------------------------------------
2021-11-21 00:01:36,034 INFO     ---------------------- Creating code2vec model ----------------------
2021-11-21 00:01:36,034 INFO     ---------------------------------------------------------------------
2021-11-21 00:01:36,034 INFO     ---------------------------------------------------------------------
2021-11-21 00:01:36,034 INFO     Checking number of examples ...
2021-11-21 00:01:36,034 INFO     ---------------------------------------------------------------------
2021-11-21 00:01:36,034 INFO     ----------------- Configuration - Hyper Parameters ---

In [ ]:
with open('code_vector_results.txt','r') as firstfile, open('code_vector_results_Dataset.txt','w') as secondfile:
    for line in firstfile:
             secondfile.write(line)

In [ ]:
import pandas as pd

vectors_table = pd.read_csv('code_vector_results_Dataset.txt', sep=" ", header=None)

In [ ]:
vectors_table.shape

(11, 384)

In [ ]:
vectors_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
0,0.547876,-0.428099,-0.765099,-0.889971,0.365057,-0.718065,-0.621206,-0.112487,-0.564392,0.744526,0.677504,-0.409114,0.117288,0.736798,0.081772,0.405668,0.320441,0.200680,-0.212435,0.482292,-0.376135,0.810500,0.590767,0.374566,0.028311,-0.477916,0.030712,0.327910,-0.537569,0.196479,-0.164221,-0.186790,0.869429,-0.400327,-0.259481,0.608660,0.197638,0.320440,0.451429,-0.664520,...,0.413078,-0.191485,-0.559087,0.199954,0.483585,-0.526764,-0.743412,0.618677,0.333758,0.581103,-0.495238,0.817814,0.250623,0.422336,0.794443,-0.258847,0.175929,0.112028,0.568174,0.308901,0.067249,0.516215,0.486162,0.478737,-0.304103,0.450481,0.268959,-0.289627,-0.394742,-0.535584,-0.892085,-0.457885,0.586763,0.620405,-0.529608,-0.297156,-0.323836,-0.772070,0.705877,-0.278182
1,-0.393870,-0.586053,0.736040,-0.589478,-0.351709,-0.329741,0.026044,-0.796471,0.106537,0.657664,-0.395093,0.439415,-0.938105,-0.541834,-0.315735,-0.496385,-0.213699,-0.327903,-0.282127,0.578761,0.674463,-0.101693,-0.041054,0.258532,-0.373113,-0.117707,0.620878,-0.299635,-0.240722,0.472117,0.402125,0.533322,0.826094,0.265298,-0.449392,0.599325,0.176293,-0.365616,-0.249911,-0.365623,...,-0.213229,-0.228509,-0.670937,0.418265,0.061652,-0.102603,0.044412,0.574532,-0.048377,0.048625,-0.585278,-0.523637,-0.404361,-0.207068,-0.229009,0.435752,0.131924,-0.506990,-0.166555,-0.889225,-0.669139,-0.527303,-0.277920,0.052277,-0.037778,0.215471,0.042971,0.502665,-0.374361,-0.018751,-0.588130,-0.392676,0.606352,0.448390,-0.268493,0.511602,-0.187238,0.363410,0.070423,-0.001937
2,0.792138,0.564390,-0.198197,0.286289,0.331912,0.230352,-0.939581,-0.069405,-0.007876,-0.031256,-0.548991,0.171505,-0.977851,0.258483,0.458622,-0.517177,-0.416740,-0.606993,0.847128,0.806758,-0.312751,-0.059203,-0.425383,-0.594346,-0.785002,-0.053693,0.435552,0.649010,-0.240544,0.695405,-0.167458,0.426097,0.320566,0.912065,-0.782952,-0.307847,-0.648880,0.012001,0.598566,0.039723,...,0.465556,-0.691677,-0.280080,0.675130,0.270419,0.152734,-0.720020,-0.198405,-0.292759,-0.422995,-0.729123,-0.450675,0.431033,-0.156759,0.140659,0.812529,0.781918,-0.548671,0.430750,-0.818671,0.752709,-0.671274,-0.948454,0.703342,-0.296479,0.548227,-0.239007,0.489333,0.147179,-0.014371,-0.594327,-0.888664,-0.817479,0.076598,0.197819,-0.837808,-0.072704,-0.343243,0.367184,0.329934
3,-0.415123,0.777246,-0.481895,-0.290132,0.232597,-0.009452,-0.153714,-0.837997,0.214122,0.052524,-0.136735,-0.058759,-0.328301,0.127833,-0.113257,-0.259027,-0.389659,0.399817,0.686316,-0.005735,0.593631,0.613766,0.596835,-0.111559,-0.365280,0.627323,0.815119,-0.956300,0.065312,0.769338,-0.892535,-0.025827,0.287555,0.726556,-0.408964,0.058226,-0.666526,0.445379,-0.037253,-0.580754,...,0.195375,0.151023,-0.636749,0.822147,0.366405,0.402009,-0.646936,-0.279179,0.113263,-0.594862,-0.561509,-0.839229,-0.856227,0.016801,0.282556,0.827272,-0.825395,-0.755045,0.305070,-0.852639,0.231220,-0.187792,-0.897122,0.738975,-0.474658,0.694529,-0.424849,0.596559,-0.483752,-0.521559,0.124698,-0.075176,-0.469774,0.029287,0.152552,0.325270,-0.904016,0.751905,0.639023,0.225380
4,0.280294,0.704815,-0.296998,-0.723796,-0.614707,-0.249094,0.549011,-0.631891,-0.330863,0.686149,-0.193997,0.268324,0.367977,0.644378,-0.360696,-0.657342,-0.899150,0.636666,0.335998,0.892431,0.161619,-0.366163,-0.051186,-0.701309,-0.711373,0.150705,-0.074853,-0.541933,0.671517,0.819234,0.600302,-0.029840,0.937720,0.251853,0.309857,-0.349089,0.188262,-0.531554,-0.150845,-0.799906,...,0.083746,-0.486427,0.360718,0.909858,-0.092168,-0.592927,-0.825291,0.566152,-0.548336,0.265852,-0.518838,0.799782,-0.475256,0.753984,-0.676743,0.343805,-0.410903,0.263201,0.828995,0.752696,0.540614,0.256801,-0.509119,0.704588,-0.524446,-0.602871,0.245499,0.525697,-0.1

In [ ]:
!rm  code_vector_results.txt

In [ ]:
with open('oracle2.java','r', encoding='utf-8') as firstfile, open('Input.java','w', encoding='utf-8') as secondfile:
  for i,line in enumerate(firstfile):
    secondfile.write(line)

In [ ]:
!python code2vec.py --load models/java14_model/saved_model_iter8.release --predict --export_code_vectors

2021-11-20 23:45:40.607033: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-20 23:45:40,608 INFO     
2021-11-20 23:45:40,608 INFO     
2021-11-20 23:45:40,608 INFO     ---------------------------------------------------------------------
2021-11-20 23:45:40,608 INFO     ---------------------------------------------------------------------
2021-11-20 23:45:40,608 INFO     ---------------------- Creating code2vec model ----------------------
2021-11-20 23:45:40,608 INFO     ---------------------------------------------------------------------
2021-11-20 23:45:40,608 INFO     ---------------------------------------------------------------------
2021-11-20 23:45:40,608 INFO     Checking number of examples ...
2021-11-20 23:45:40,608 INFO     ---------------------------------------------------------------------
2021-11-20 23:45:40,608 INFO     ----------------- Configuration - Hyper Parameters ---

In [ ]:
with open('code_vector_results.txt','r') as firstfile, open('code_vector_results_Dataset_oracle.txt','w') as secondfile:
    for line in firstfile:
             secondfile.write(line)

In [ ]:
import pandas as pd

vectors_table_oracle = pd.read_csv('code_vector_results_Dataset_oracle.txt', sep=" ", header=None)

In [ ]:
vectors_table_oracle.shape

(11, 384)

In [ ]:
vectors_table_oracle

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
0,0.471680,-0.516365,-0.657480,-0.932573,0.287567,-0.737424,-0.648982,-0.020654,-0.595898,0.761477,0.646006,-0.563880,0.143725,0.827329,0.054240,0.380723,0.291303,0.203693,-0.272214,0.377607,-0.399777,0.816747,0.617621,0.414975,0.118785,-0.636793,0.035979,0.262265,-0.650385,0.210000,-0.191746,-0.205712,0.931943,-0.420916,-0.370063,0.531738,0.156878,0.318736,0.435917,-0.727338,...,0.364758,-0.099230,-0.508555,0.134038,0.467315,-0.613588,-0.700098,0.580887,0.258641,0.690730,-0.417714,0.757091,0.285735,0.439322,0.793106,-0.293210,0.297228,0.102643,0.700336,0.245087,0.100693,0.474345,0.456871,0.463752,-0.418729,0.381830,0.405321,-0.229034,-0.361123,-0.520641,-0.789328,-0.467663,0.680243,0.630738,-0.508821,-0.167516,-0.330522,-0.809463,0.749597,-0.432647
1,-0.422070,-0.626581,0.800481,-0.383711,-0.409266,-0.190185,-0.118580,-0.807159,0.212693,0.669215,-0.386556,0.484808,-0.894450,-0.639908,-0.137642,-0.511264,-0.278804,-0.541103,-0.430454,0.608733,0.688446,-0.187630,-0.259683,0.122411,-0.448696,-0.339991,0.612680,-0.153397,-0.315443,0.552944,0.451914,0.600722,0.858147,0.112409,-0.548577,0.308811,0.319531,-0.308356,-0.333196,-0.380056,...,-0.089191,-0.300925,-0.703473,0.488423,0.056138,-0.205614,0.129919,0.636353,0.106158,0.075234,-0.766572,-0.599865,-0.301131,-0.329106,-0.333064,0.515498,0.315397,-0.074667,-0.066490,-0.909543,-0.707938,-0.589509,-0.388968,0.048430,-0.067447,0.309465,-0.067265,0.536332,-0.427821,0.092350,-0.612591,-0.454600,0.353718,0.164901,-0.134249,0.548170,-0.228842,0.388013,0.069291,0.091488
2,0.962921,0.590914,-0.632839,-0.082057,-0.693379,0.116276,-0.944341,-0.493179,-0.433139,0.419771,0.674956,0.822421,-0.987430,0.661274,0.320126,0.111673,-0.945886,-0.762055,0.845374,0.961567,-0.746132,0.558537,0.528456,-0.837220,-0.942970,0.129270,0.132551,0.869100,0.367952,0.712691,-0.063358,-0.303467,0.328153,0.821474,-0.791453,-0.330411,-0.338439,-0.841368,0.400586,0.261746,...,0.342790,0.010578,0.332949,0.815425,0.481266,-0.545401,-0.132935,-0.927359,-0.637484,-0.162875,-0.624239,0.808325,0.805709,0.650057,-0.872042,0.985273,0.857487,0.384252,0.930252,0.275932,0.638085,-0.672626,-0.645945,0.876747,-0.154924,0.781734,0.360532,0.181125,0.308290,-0.696509,-0.828276,-0.560331,-0.874069,-0.731246,-0.258806,-0.549943,0.893558,-0.702688,-0.692233,0.286247
3,-0.584640,0.860990,-0.596188,-0.822706,-0.023702,-0.510951,0.459234,-0.911971,-0.221312,0.744288,0.639749,0.770056,0.490657,0.113252,-0.346032,0.273337,-0.879963,0.428313,0.572257,-0.032692,0.557356,0.805144,0.520105,0.355371,-0.327008,0.110338,-0.199282,-0.781265,-0.148023,0.999579,-0.963852,-0.444602,0.976934,0.196770,0.178159,0.858099,-0.235793,0.479717,-0.308389,-0.782788,...,-0.037084,0.848150,-0.956143,0.997818,0.234494,-0.026279,-0.491464,-0.678226,0.019590,-0.303849,-0.182083,-0.600595,-0.976498,0.654331,0.433322,0.878413,-0.867750,-0.475296,0.541182,-0.018677,-0.309926,0.047641,-0.172300,0.890616,-0.398873,0.943587,0.209834,0.702103,0.012385,-0.478406,-0.278419,0.763884,-0.679973,-0.744001,0.580736,0.195924,-0.445983,0.781044,-0.127270,0.360675
4,-0.561648,-0.192985,0.674465,0.469106,-0.499389,-0.047519,-0.688528,-0.893421,-0.653699,0.081300,0.447105,-0.185672,0.027993,0.023862,0.713418,0.384428,-0.857318,-0.108337,-0.495170,0.054614,0.691706,-0.114396,-0.432822,-0.956377,-0.697179,0.406576,-0.448274,0.465764,0.373245,0.795053,0.771939,-0.202541,0.657863,-0.387152,0.679614,-0.367859,0.311073,-0.349512,0.108434,-0.954096,...,-0.892308,-0.863297,-0.688371,0.721373,0.595323,-0.767628,0.213365,0.413374,0.662663,-0.428235,-0.683020,0.608671,-0.606517,0.339280,0.332258,-0.379194,-0.191383,-0.100445,0.220400,-0.072915,0.561536,0.034406,-0.788493,-0.173120,-0.235815,-0.080310,-0.417075,0.386886,-0.74292

In [ ]:
from scipy import spatial
results = list()
for i in range(0,len(vectors_table)):
  result = 1 - spatial.distance.cosine(vectors_table_oracle[i], vectors_table[i])
  results.append(result)

In [ ]:
results

[0.8329273720639238,
 0.8680002769677292,
 0.6554136193865654,
 0.8502736225960171,
 0.4494033850051975,
 0.9520786606977608,
 0.5922058379825798,
 0.8820531550835908,
 0.8184970385803418,
 0.8147295335309117,
 0.03755233087539178]